In [ ]:
!pip install sentence_transformers


In [ ]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from google.colab import files

In [ ]:
myfile = files.upload()

Saving A1_reply.txt to A1_reply.txt


In [ ]:
f = "./A1_reply"

reply_txt = open('A1_reply.txt', 'r')
reply= reply_txt.readline()
print(reply)

“I find ways to make money” answers everything. Let him defend Ukraine, then after the war’s over assess his contribution, and whether he has learned to conduct business ethically.



In [ ]:
n_gram_range = (2, 3)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([reply])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 25
trigram 다섯개만 출력 : ['answers let' 'answers let defend' 'assess contribution'
 'assess contribution learned' 'business ethically']


In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([reply])
candidate_embeddings = model.encode(candidates)

In [ ]:
top_n = 5
diversity = 0.7

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
candidates_idx = [i for i in range(len(candidates)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
for _ in range(top_n - 1):
   candidate_similarities = word_doc_similarity[candidates_idx, :]
   target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
   mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
   mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
   keywords_idx.append(mmr_idx)
   candidates_idx.remove(mmr_idx)

cos_sim_keywords = [candidates[idx] for idx in keywords_idx]

In [ ]:
keyword_print = open("article_keywords.txt", 'w')
for i in range(len(cos_sim_keywords)):
   keyword_print.write("#" + cos_sim_keywords[i] + "     ")
keyword_print.close()
#저장할때 프로젝트 파일 내부에 저장해주기